In [ ]:
from math import nan, isnan

from pandas import read_excel, DataFrame, to_datetime, concat, Categorical, crosstab, Series
from scipy.stats import pearsonr, chi2_contingency
from scipy.stats.contingency import Chi2ContingencyResult
from seaborn import set_theme, displot, move_legend

In [ ]:
df: DataFrame = read_excel("../data/06-study.xlsx", sheet_name="07 Survey", header=0)
df.rename(columns={
    "V1": "survey_id",
    "Zeitstempel": "datetime",
    "FrageNr": "qid",
    "Fachlichpersönlich": "background",
    "ObjektA": "object1",
    "ObjektB": "object2",
    "HastdudazuVorkenntnisse": "prior_knowledge",
    "WiewürdestdudichohneweitergehendeRechercheentscheiden": "decision_before_search_raw",
    "Wiesicherbistdudirdamit": "confidence_before_search",
    "HastdudiesesDokumentgelesen": "has_read_document1",
    "HastdudiesesDokumentgelesen_A": "has_read_document2",
    "HastdudiesesDokumentgelesen_B": "has_read_document3",
    "HastdudiesesDokumentgelesen_C": "has_read_document4",
    "WieentscheidestdudichnachdemdudieseDokumentegelesenhast": "decision_after_search_raw",
    "Wiesicherbistdudirjetztdamit": "confidence_after_search",
    "WelchesDokumentehathabendeineEntscheidungbeeinflusst": "influencing_documents",
    "WelchedieserAussagentrifftzu": "statements",
    "Relevanz_Dok1": "relevance_document1",
    "Relevanz_Dok2": "relevance_document2",
    "Relevanz_Dok3": "relevance_document3",
    "Relevanz_Dok4": "relevance_document4",
    "Haltung_Dok1": "stance_document1",
    "Haltung_Dok2": "stance_document2",
    "Haltung_Dok3": "stance_document3",
    "Haltung_Dok4": "stance_document4",
    "Qualität_Dok1": "quality_score_document1",
    "Qualität_Dok2": "quality_score_document2",
    "Qualität_Dok3": "quality_score_document3",
    "Qualität_Dok4": "quality_score_document4",
    "Qualität_Gesamt": "quality_score",
    "Entscheidung1A": "decision_before_search_object1",
    "Entscheidung1B": "decision_before_search_object2",
    "Entscheidung2A": "decision_after_search_object1",
    "Entscheidung2B": "decision_after_search_object2",
    "Beeinflussung1": "influence_document1",
    "Beeinflussung2": "influence_document2",
    "Beeinflussung3": "influence_document3",
    "Beeinflussung4": "influence_document4",
    "AussageDieRecherchehatmichinmeinerMeinungbestätigt": "statement_research_confirmed",
    "AussageIchkannmichmithilfedieserSuchergebnissebesserentscheiden": "statement_research_helped",
    "AussageDurchdieRecherchehabeichetwasNeueszudemThemagelernt": "statement_research_learned",
    "AussageDieSuchergebnissehabenmirnichtgeholfen": "statement_research_not_helped",
    "AussageIchwürdeandieserStellenochweiterrecherchieren": "statement_research_continue",
    "EntscheidungVorher": "stance_before_search",
    "EntscheidungVorherA": "stance_before_search_object1",
    "EntscheidungVorherB": "stance_before_search_object2",
    "EntscheidungNachher": "stance_after_search",
    "EntscheidungNachherA": "stance_after_search_object1",
    "EntscheidungNachherB": "stance_after_search_object2",
    "EntscheidungVorherX": "stance_before_search_none",
    "EntscheidungNachherX": "stance_after_search_none",
}, inplace=True)
df["background"] = df["background"].map({
    "fachlich": "factual",
    "persönlich": "subjective",
}).astype("category")
for doc in [1, 2, 3, 4]:
    df[f"has_partially_read_document{doc}"] = df[f"has_read_document{doc}"].map({
        "Ja": True,
        "Überflogen": True,
        "Nein": False,
    })
    df[f"has_fully_read_document{doc}"] = df[f"has_read_document{doc}"].map({
        "Ja": True,
        "Überflogen": False,
        "Nein": False,
    })
    df[f"stance_direction_document{doc}"] = df[f"stance_document{doc}"].map({
        1: "object1",
        2: "object1",
        3: "neutral",
        4: "object2",
        5: "object2",
        6: nan,
        nan: nan,
    }, na_action="ignore")
    df[f"stance_strength_document{doc}"] = df[f"stance_document{doc}"].map({
        1: 2,
        2: 1,
        3: 0,
        4: 1,
        5: 2,
        6: nan,
        nan: nan,
    }, na_action="ignore")
df["prior_knowledge"] = df["prior_knowledge"].map({
    "Ja": True,
    "Nein": False,
})
df["datetime"] = to_datetime(df["datetime"], format="%d.%m.%Y %H:%M:%S", utc=True)
df.drop(columns=[
    "datetime",
    "object1",
    "object2",
    "decision_before_search_raw",
    "decision_after_search_raw",
    "influencing_documents",
    "statements",
    "stance_before_search",
    "stance_before_search_object1",
    "stance_before_search_object2",
    "stance_before_search_none",
    "stance_after_search",
    "stance_after_search_object1",
    "stance_after_search_object2",
    "stance_after_search_none",
    "has_read_document1",
    "has_read_document2",
    "has_read_document3",
    "has_read_document4",
], inplace=True)
for col in [
    "decision_before_search_object1",
    "decision_before_search_object2",
    "decision_after_search_object1",
    "decision_after_search_object2",
    "influence_document1",
    "influence_document2",
    "influence_document3",
    "influence_document4",
    "statement_research_confirmed",
    "statement_research_helped",
    "statement_research_learned",
    "statement_research_not_helped",
    "statement_research_continue",
    "relevance_document1",
    "relevance_document2",
    "relevance_document3",
    "relevance_document4",
]:
    df[col] = df[col].astype(bool)
df["decision_before_search_none"] = ~df["decision_before_search_object1"] & ~df["decision_before_search_object2"]
df["decision_after_search_none"] = ~df["decision_after_search_object1"] & ~df["decision_after_search_object2"]
df["confidence_delta"] = df["confidence_after_search"] - df["confidence_before_search"]
df["decision_unchanged"] = (
        df["decision_before_search_object1"].eq(df["decision_after_search_object1"]) &
        df["decision_before_search_object2"].eq(df["decision_after_search_object2"]) &
        df["decision_before_search_none"].eq(df["decision_after_search_none"])
)
df["decision_changed"] = ~df["decision_unchanged"]
df["num_influencing_documents"] = df[[
    "influence_document1",
    "influence_document2",
    "influence_document3",
    "influence_document4",
]].sum(axis="columns")
df["relevance"] = df[[
    "relevance_document1",
    "relevance_document2",
    "relevance_document3",
    "relevance_document4",
]].mean(axis="columns")
df

In [ ]:
df_doc = concat([
    DataFrame([
        {
            "survey_id": row["survey_id"],
            "qid": row["qid"],
            "background": row["background"],
            "rank": rank,
            "decision_before_search_object1": row["decision_before_search_object1"],
            "decision_before_search_object2": row["decision_before_search_object2"],
            "decision_before_search_none": row["decision_before_search_none"],
            "confidence_before_search": row["confidence_before_search"],
            "decision_after_search_object1": row["decision_after_search_object1"],
            "decision_after_search_object2": row["decision_after_search_object2"],
            "decision_after_search_none": row["decision_after_search_none"],
            "confidence_after_search": row["confidence_after_search"],
            "decision_unchanged": row["decision_unchanged"],
            "decision_changed": row["decision_changed"],
            "confidence_delta": row["confidence_delta"],
            "has_fully_read_document": row[f"has_fully_read_document{rank}"],
            "has_partially_read_document": row[f"has_partially_read_document{rank}"],
            "influence_document": row[f"influence_document{rank}"],
            "prior_knowledge": row["prior_knowledge"],
            "quality_score": row["quality_score"],
            "quality_score_document": row[f"quality_score_document{rank}"],
            "relevance_document": row[f"relevance_document{rank}"],
            "stance_document": row[f"stance_document{rank}"],
            "stance_direction_document": row[f"stance_direction_document{rank}"],
            "stance_strength_document": row[f"stance_strength_document{rank}"],
            "statement_research_confirmed": row["statement_research_confirmed"],
            "statement_research_continue": row["statement_research_continue"],
            "statement_research_helped": row["statement_research_helped"],
            "statement_research_learned": row["statement_research_learned"],
            "statement_research_not_helped": row["statement_research_not_helped"],
        }
        for rank in [1, 2, 3, 4]
    ])
    for _, row in df.iterrows()
])
df_doc

## Descriptive statistics

In [ ]:
df_doc["background"].value_counts(dropna=False)

In [ ]:
df_doc[["has_fully_read_document", "has_partially_read_document"]].value_counts(dropna=False)

In [ ]:
df_doc[
    ["decision_before_search_none", "decision_before_search_object1", "decision_before_search_object2"]].value_counts(
    dropna=False)

In [ ]:
df_doc[["decision_after_search_none", "decision_after_search_object1", "decision_after_search_object2"]].value_counts(
    dropna=False)

In [ ]:
df_doc[["decision_before_search_none", "decision_after_search_none"]].value_counts(dropna=False)

In [ ]:
df_doc["decision_changed"].value_counts(dropna=False)

## Histograms

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df.copy()
df_plot = df_plot[df_plot["confidence_delta"].notna()]
df_plot["confidence_changed"] = df_plot["confidence_delta"].map(
    lambda x: "no change" if x == 0 else "more confident" if x > 0 else "less confident")
df_plot["confidence_changed"] = Categorical(
    df_plot["confidence_changed"],
    categories=["less confident", "no change", "more confident"],
    ordered=True,
)
df_plot["background"] = Categorical(
    df_plot["background"],
    categories=["factual", "subjective"],
    ordered=True,
)
df_plot.sort_values(["confidence_changed", "background"], inplace=True)
plot = displot(
    df_plot,
    x="background",
    hue="confidence_changed",
    multiple="dodge",
    aspect=1.7,
    height=2.5,
    legend=False,
    shrink=0.9,
)
plot.set_xlabels(label="Background")
plot.savefig("../data/figures/histogram-confidence-change-background.pdf")
plot

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df.copy()
df_plot = df_plot[df_plot["confidence_delta"].notna()]
df_plot["confidence_changed"] = df_plot["confidence_delta"].map(
    lambda x: "no change" if x == 0 else "more confident" if x > 0 else "less confident")
df_plot["confidence_changed"] = Categorical(
    df_plot["confidence_changed"],
    categories=["less confident", "no change", "more confident"],
    ordered=True,
)
df_plot["decision_changed"] = df_plot["decision_changed"].map(
    lambda x: "change" if x else "no change")
df_plot["decision_changed"] = Categorical(
    df_plot["decision_changed"],
    categories=["no change", "change"],
    ordered=True,
)
df_plot.sort_values(["decision_changed", "confidence_changed"], inplace=True)
plot = displot(
    df_plot,
    x="decision_changed",
    hue="confidence_changed",
    multiple="dodge",
    aspect=1.7,
    height=2.5,
    shrink=0.9,
)
move_legend(
    plot,
    title="Confidence",
    loc="upper right",
    bbox_to_anchor=(0.69, 1),
)
plot.set_xlabels(label="Decision")
plot.savefig("../data/figures/histogram-confidence-change-decision-change.pdf")
plot

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df_doc.copy()
plot = displot(
    df_plot,
    x="quality_score_document",
    hue="influence_document",
    multiple="fill",
)
plot.set_xlabels("Quality")
plot.set_ylabels("Proportion")
plot.legend.set_title("Influence")
plot

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df_doc.copy()
df_plot = df_plot[df_plot["stance_document"].notna()]
plot = displot(
    df_plot,
    x="stance_document",
    hue="influence_document",
    multiple="fill",
)
plot.set_xlabels("Stance strength")
plot.set_ylabels("Proportion")
plot.legend.set_title("Influence")
plot

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df_doc.copy()
# df_plot["relevance_document"] = df_plot["relevance_document"].map(
#     lambda x: "relevant" if x else "irrelevant")
plot = displot(
    df_plot,
    x="relevance_document",
    hue="influence_document",
    multiple="fill",
)
plot.set_xlabels("Relevance")
plot.set_ylabels("Proportion")
plot.legend.set_title("Influence")
plot

In [ ]:
set_theme(
    style="ticks",
    palette="colorblind",
    font_scale=1.0,
    font="sans-serif",
)
df_plot = df.copy()
df_plot.sort_values("num_influencing_documents", inplace=True)
df_plot["num_influencing_documents"] = df_plot["num_influencing_documents"].astype(str)
plot = displot(
    df_plot,
    x="num_influencing_documents",
    hue="background",
    multiple="stack",
    aspect=2,
    height=2.5,
    shrink=0.7,
)
plot.set_xlabels("# Influencing documents")
move_legend(
    plot,
    title="Background",
    loc="upper right",
    bbox_to_anchor=(0.77, 0.9),
)
plot.savefig("../data/figures/histogram-num-influencing-documents.pdf")
plot

## Significance tests

In [ ]:
def chi2_to_tex(chi2_result: Chi2ContingencyResult, alpha: float, p_value_digits: int = 3) -> str:
    statistic, p_value, dof, _ = chi2_result
    is_significant = p_value < alpha
    statistic_tex = r"\chi^2(" + f"{dof}" + r")" + f"={statistic:.2f}"
    min_p_value = pow(10, -p_value_digits)
    p_value_tex = format(max(p_value, min_p_value), f".{p_value_digits}f")
    significance_tex = r"p" + ("=" if p_value >= min_p_value else "<") + p_value_tex
    if is_significant:
        statistic_tex = r"\bm{" + statistic_tex + r"}"
        significance_tex = r"\bm{" + significance_tex + r"}"
    tex = r"\(" + statistic_tex + r"\), \(" + significance_tex + r"\)"
    if is_significant:
        tex = r"\textbfn{" + tex + r"}"
    return tex

In [ ]:
def diff_expected_freq_tex(row: Series, col: str, index: str, i: int, chi2_result: Chi2ContingencyResult) -> str:
    if index == r"\(\sum\)":
        return ""
    col_index = row.index.get_loc(col)
    diff_expected_freq = (row[col] - chi2_result.expected_freq[i,col_index]).round()
    diff_expected_freq_sign = r"\(+\)" if diff_expected_freq > 0 else r"--" if diff_expected_freq < 0 else r"\(\pm\)"
    diff_expected_freq_tex = f"{diff_expected_freq_sign}{abs(diff_expected_freq):.0f}"
    diff_expected_freq_tex = r"\secondary{(" + diff_expected_freq_tex + r")}"
    return diff_expected_freq_tex

In [ ]:
# Copy-pasted from the quality assessments evaluation notebook (median).
quality_threshold = 0.56969696969697
label_low_quality = r"low quality~\secondary{(\(<" + f"{quality_threshold:.2f}" + r"\))}"
label_high_quality = r"high quality~\secondary{(\(\geq" + f"{quality_threshold:.2f}" + r"\))}"
quality_threshold

In [ ]:
initial_confidence_threshold = df["confidence_before_search"].median()
label_low_initial_confidence = r"low confidence~\secondary{(\(<" + f"{initial_confidence_threshold:.0f}" + r"\))}"
label_high_initial_confidence = r"high confidence~\secondary{(\(\geq" + f"{initial_confidence_threshold:.0f}" + r"\))}"
initial_confidence_threshold

In [ ]:
final_confidence_threshold = df["confidence_after_search"].median()
label_low_final_confidence = r"low confidence~\secondary{(\(<" + f"{final_confidence_threshold:.0f}" + r"\))}"
label_high_final_confidence = r"high confidence~\secondary{(\(\geq" + f"{final_confidence_threshold:.0f}" + r"\))}"
final_confidence_threshold

In [ ]:
df_test = df_doc.copy()
df_test = df_test[df_test["has_partially_read_document"]]
df_test["influence_document"] = df_test["influence_document"].map(
    lambda x: "influence" if x else "no influence")
df_test["influence_document"] = Categorical(
    df_test["influence_document"],
    categories=["no influence", "influence"],
    ordered=True,
)
df_test["quality_document"] = df_test["quality_score_document"].map(
    lambda x: label_high_quality if x >= quality_threshold else label_low_quality)
df_test["quality_document"] = Categorical(
    df_test["quality_document"],
    categories=[label_low_quality, label_high_quality],
    ordered=True,
)
df_test["relevance_document"] = df_test["relevance_document"].map(
    lambda x: "relevant" if x else "not relevant")
df_test["relevance_document"] = Categorical(
    df_test["relevance_document"],
    categories=["not relevant", "relevant"],
    ordered=True,
)
df_test["relevance_quality_document"] = [
    row["relevance_document"] + ", " + row["quality_document"]
    for _, row in df_test.iterrows()
]
df_test["relevance_quality_document"] = Categorical(
    df_test["relevance_quality_document"],
    categories=[f"not relevant, {label_low_quality}", f"not relevant, {label_high_quality}", f"relevant, {label_low_quality}", f"relevant, {label_high_quality}"],
    ordered=True,
)
df_test["stance_strength_document"] = df_test["stance_strength_document"].map({
    0: "no stance",
    1: "weak stance",
    2: "strong stance",
})
df_test["stance_strength_document"] = Categorical(
    df_test["stance_strength_document"],
    categories=["no stance", "weak stance", "strong stance"],
    ordered=True,
)
df_test["initial_confidence"] = df_test["confidence_before_search"].map(
    lambda x: label_high_initial_confidence if x >= initial_confidence_threshold else label_low_initial_confidence)
df_test["initial_confidence"] = Categorical(
    df_test["initial_confidence"],
    categories=[label_low_initial_confidence, label_high_initial_confidence],
    ordered=True,
)
df_test["final_confidence"] = df_test["confidence_after_search"].map(
    lambda x: label_high_final_confidence if x >= final_confidence_threshold else label_low_final_confidence)
df_test["final_confidence"] = Categorical(
    df_test["final_confidence"],
    categories=[label_low_final_confidence, label_high_final_confidence],
    ordered=True,
)
df_test["initial_confidence_low_quality"] = df_test["initial_confidence"].where(
    df_test["quality_document"] == label_low_quality)
df_test["rank"] = "rank~" + df_test["rank"].astype(str)
df_test

In [ ]:
contingency_quality = crosstab(
    index=df_test["quality_document"],
    columns=df_test["influence_document"],
)
contingency_full_quality = crosstab(
    index=df_test["quality_document"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_quality

In [ ]:
chi2_quality = chi2_contingency(contingency_full_quality)
chi2_quality

In [ ]:
contingency_relevance = crosstab(
    index=df_test["relevance_document"],
    columns=df_test["influence_document"],
)
contingency_full_relevance = crosstab(
    index=df_test["relevance_document"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_relevance

In [ ]:
chi2_relevance = chi2_contingency(contingency_relevance)
chi2_relevance

In [ ]:
contingency_quality_quality_relevance = crosstab(
    index=df_test["relevance_quality_document"],
    columns=df_test["influence_document"],
)
contingency_full_quality_relevance = crosstab(
    index=df_test["relevance_quality_document"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_quality_relevance

In [ ]:
chi2_quality_relevance = chi2_contingency(contingency_full_quality_relevance)
chi2_quality_relevance

In [ ]:
contingency_stance_strength = crosstab(
    index=df_test["stance_strength_document"],
    columns=df_test["influence_document"],
)
contingency_full_stance_strength = crosstab(
    index=df_test["stance_strength_document"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_stance_strength

In [ ]:
chi2_stance_strength = chi2_contingency(contingency_stance_strength)
chi2_stance_strength

In [ ]:
contingency_initial_confidence_low_quality = crosstab(
    index=df_test["initial_confidence_low_quality"],
    columns=df_test["influence_document"],
)
contingency_full_initial_confidence_low_quality = crosstab(
    index=df_test["initial_confidence_low_quality"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_initial_confidence_low_quality

In [ ]:
chi2_initial_confidence_low_quality = chi2_contingency(contingency_initial_confidence_low_quality)
chi2_initial_confidence_low_quality

In [ ]:
contingency_rank = crosstab(
    index=df_test["rank"],
    columns=df_test["influence_document"],
)
contingency_full_rank = crosstab(
    index=df_test["rank"],
    columns=df_test["influence_document"],
    margins=True,
)
contingency_full_rank

In [ ]:
chi2_rank = chi2_contingency(contingency_rank)
chi2_rank

In [ ]:
tests = [
    (r"Quality", contingency_full_quality, chi2_quality),
    (r"Relevance", contingency_full_relevance, chi2_relevance),
    (r"Quality\,\(\times\)\,relevance", contingency_full_quality_relevance, chi2_quality_relevance),
    (r"Stance strength", contingency_full_stance_strength, chi2_stance_strength),
    (r"Initial confidence (for low-quality docs.)", contingency_full_initial_confidence_low_quality, chi2_initial_confidence_low_quality),
    (r"Ranking position", contingency_full_rank, chi2_rank),
]
alpha = 0.05
correction = 1
alpha_corrected = alpha / correction  # Bonferroni correction across outcomes
# TODO: Indicate change compared to expected frequencies.
print(r"\begin{tabular}{@{}l@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{}}")
print(r"  \toprule")
print(r"  \textbf{Predictor} & \multicolumn{5}{@{}c@{}}{\textbf{Document influence}} \\")
print(r"  & \multicolumn{2}{@{}l}{No influence} & \multicolumn{2}{l}{Influence} & \(\sum\) \\")
for variable_name, contingency, chi2_result in tests:
    print(r"  \midrule")
    print(r"  \textit{" + variable_name + r"} & \multicolumn{5}{@{}c@{}}{" +
          chi2_to_tex(chi2_result, alpha_corrected) + r"} \\")
    print(r"  \midrule")
    for i, (index, row) in enumerate(contingency.iterrows()):
        if isinstance(index, tuple):
            index = ", ".join(index)
        elif "All" in index:
            index = r"\(\sum\)"
        columns = [
            index,
            f"{row['no influence']}",
            diff_expected_freq_tex(row, "no influence", index, i, chi2_result),
            f"{row['influence']}",
            diff_expected_freq_tex(row, "influence", index, i, chi2_result),
            f"{row['All']}",
        ]
        print(r"  " + r" & ".join(columns) + r" \\")
print(r"  \bottomrule")
print(r"\end{tabular}")

In [ ]:
df_test2 = df.copy()
df_test2["decision_changed"] = df_test2["decision_changed"].map(
    lambda x: "changed decision" if x else "same decision")
df_test2["decision_changed"] = Categorical(
    df_test2["decision_changed"],
    categories=["same decision", "changed decision"],
    ordered=True,
)
df_test2["final_confidence"] = df_test2["confidence_after_search"].map(
    lambda x: label_high_final_confidence if x >= final_confidence_threshold else label_low_final_confidence)
df_test2["final_confidence"] = Categorical(
    df_test2["final_confidence"],
    categories=[label_low_final_confidence, label_high_final_confidence],
    ordered=True,
)
df_test2["confidence_changed"] = df_test2["confidence_delta"].map(
    lambda x: nan if isnan(x) else "less confident" if x < 0 else "more confident" if x > 0 else "same confidence")
df_test2["confidence_changed"] = Categorical(
    df_test2["confidence_changed"],
    categories=["less confident", "same confidence", "more confident"],
    ordered=True,
)
df_test2["quality"] = df_test2["quality_score"].map(
    lambda x: label_high_quality if x >= quality_threshold else label_low_quality)
df_test2["quality"] = Categorical(
    df_test2["quality"],
    categories=[label_low_quality, label_high_quality],
    ordered=True,
)
df_test2["background"] = Categorical(
    df_test2["background"],
    categories=["factual", "subjective"],
    ordered=True,
)
for col in (
        "statement_research_confirmed",
        "statement_research_helped",
        "statement_research_not_helped",
        "statement_research_learned",
        "statement_research_continue",
            ):
    df_test2[col]=    df_test2[col].map({True: "yes", False: "no"})
    df_test2[col] = Categorical(
        df_test2[col],
        categories=["no", "yes"],
        ordered=True,
    )
df_test2

In [ ]:
contingency_background_decision_change = crosstab(
    index=df_test2["background"],
    columns=df_test2["decision_changed"],
)
contingency_full_background_decision_change = crosstab(
    index=df_test2["background"],
    columns=df_test2["decision_changed"],
    margins=True,
)
contingency_full_background_decision_change

In [ ]:
chi2_background_decision_change = chi2_contingency(contingency_background_decision_change)
chi2_background_decision_change

In [ ]:
contingency_quality_decision_change = crosstab(
    index=df_test2["quality"],
    columns=df_test2["decision_changed"],
)
contingency_full_quality_decision_change = crosstab(
    index=df_test2["quality"],
    columns=df_test2["decision_changed"],
    margins=True,
)
contingency_full_quality_decision_change

In [ ]:
chi2_quality_decision_change = chi2_contingency(contingency_quality_decision_change)
chi2_quality_decision_change

In [ ]:
contingency_background_final_confidence = crosstab(
    index=df_test2["background"],
    columns=df_test2["final_confidence"],
)
contingency_full_background_final_confidence = crosstab(
    index=df_test2["background"],
    columns=df_test2["final_confidence"],
    margins=True,
)
contingency_full_background_final_confidence

In [ ]:
chi2_background_final_confidence = chi2_contingency(contingency_background_final_confidence)
chi2_background_final_confidence

In [ ]:
contingency_quality_final_confidence = crosstab(
    index=df_test2["quality"],
    columns=df_test2["final_confidence"],
)
contingency_full_quality_final_confidence = crosstab(
    index=df_test2["quality"],
    columns=df_test2["final_confidence"],
    margins=True,
)
contingency_full_quality_final_confidence

In [ ]:
chi2_quality_final_confidence = chi2_contingency(contingency_quality_final_confidence)
chi2_quality_final_confidence

In [ ]:
contingency_background_confidence_change = crosstab(
    index=df_test2["background"],
    columns=df_test2["confidence_changed"],
)
contingency_full_background_confidence_change = crosstab(
    index=df_test2["background"],
    columns=df_test2["confidence_changed"],
    margins=True,
)
contingency_full_background_confidence_change

In [ ]:
chi2_background_confidence_change = chi2_contingency(contingency_background_confidence_change)
chi2_background_confidence_change

In [ ]:
contingency_quality_confidence_change = crosstab(
    index=df_test2["quality"],
    columns=df_test2["confidence_changed"],
)
contingency_full_quality_confidence_change = crosstab(
    index=df_test2["quality"],
    columns=df_test2["confidence_changed"],
    margins=True,
)
contingency_full_quality_confidence_change

In [ ]:
chi2_quality_confidence_change = chi2_contingency(contingency_quality_confidence_change)
chi2_quality_confidence_change

In [ ]:
contingency_decision_confidence_change = crosstab(
    index=df_test2["decision_changed"],
    columns=df_test2["confidence_changed"],
)
contingency_full_decision_confidence_change = crosstab(
    index=df_test2["decision_changed"],
    columns=df_test2["confidence_changed"],
    margins=True,
)
contingency_full_decision_confidence_change

In [ ]:
chi2_decision_confidence_change = chi2_contingency(contingency_decision_confidence_change)
chi2_decision_confidence_change

In [ ]:
tests = [
    (r"Background", contingency_full_background_decision_change, chi2_background_decision_change, contingency_full_background_final_confidence, chi2_background_final_confidence, contingency_full_background_confidence_change, chi2_background_confidence_change),
    (r"Quality", contingency_full_quality_decision_change, chi2_quality_decision_change, contingency_full_quality_final_confidence, chi2_quality_final_confidence, contingency_full_quality_confidence_change, chi2_quality_confidence_change),
]
alpha = 0.05
correction = 3
alpha_corrected = alpha / correction  # Bonferroni correction across outcomes
# TODO: Indicate change compared to expected frequencies.
# print(r"\newlength{\largetabcolsep}")
# print(r"\setlength{\largetabcolsep}{3\tabcolsep}")
print(r"\begin{tabular}{@{}l@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{ }lr@{}}")
print(r"  \toprule")
print(r"  \textbf{Predictor} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Decision change}} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Final decision confidence}} & \multicolumn{7}{@{}c@{}}{\textbf{Decision confidence change}} \\")
print(r"  & \multicolumn{2}{@{}l}{Unchanged} & \multicolumn{2}{l}{Changed} & \(\sum\) & \multicolumn{2}{@{}l}{Low~\secondary{(\(<"+f"{final_confidence_threshold:.2f}"+r"\))}} & \multicolumn{2}{l}{High~\secondary{(\(<"+f"{final_confidence_threshold:.2f}"+r"\))}} & \(\sum\) & \multicolumn{2}{@{}l}{Decreased} & \multicolumn{2}{l}{Unchanged} & \multicolumn{2}{l}{Increased} & \(\sum\) \\")
for variable_name, contingency_decision_change, chi2_result_decision_change, contingency_final_confidence, chi2_result_final_confidence, contingency_confidence_change, chi2_result_confidence_change in tests:
    print(r"  \midrule")
    print(r"  \textit{" + variable_name + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_decision_change, alpha_corrected) + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_final_confidence, alpha_corrected) + r"} & \multicolumn{7}{@{}c@{}}{" + chi2_to_tex(chi2_result_confidence_change, alpha_corrected) + r"} \\")
    print(r"  \midrule")
    for i, ((index_decision_change, row_decision_change), (index_final_confidence, row_final_confidence), (index_confidence_change, row_confidence_change)) in enumerate(zip(contingency_decision_change.iterrows(), contingency_final_confidence.iterrows(), contingency_confidence_change.iterrows())):
        assert index_decision_change == index_final_confidence == index_confidence_change
        index = index_confidence_change
        if "All" in index:
            index = r"\(\sum\)"
        columns = [
            index,
            f"{row_decision_change['same decision']}",
            diff_expected_freq_tex(row_decision_change, "same decision", index, i, chi2_result_decision_change),
            f"{row_decision_change['changed decision']}",
            diff_expected_freq_tex(row_decision_change, "changed decision", index, i, chi2_result_decision_change),
            f"{row_decision_change['All']}",
            f"{row_final_confidence[label_low_final_confidence]}",
            diff_expected_freq_tex(row_final_confidence, label_low_final_confidence, index, i, chi2_result_final_confidence),
            f"{row_final_confidence[label_high_final_confidence]}",
            diff_expected_freq_tex(row_final_confidence, label_high_final_confidence, index, i, chi2_result_final_confidence),
            f"{row_final_confidence['All']}",
            f"{row_confidence_change['less confident']}",
            diff_expected_freq_tex(row_confidence_change, "less confident", index, i, chi2_result_confidence_change),
            f"{row_confidence_change['same confidence']}",
            diff_expected_freq_tex(row_confidence_change, "same confidence", index, i, chi2_result_confidence_change),
            f"{row_confidence_change['more confident']}",
            diff_expected_freq_tex(row_confidence_change, "more confident", index, i, chi2_result_confidence_change),
            f"{row_confidence_change['All']}",
        ]
        print(r"  " + r" & ".join(columns) + r" \\")
print(r"  \bottomrule")
print(r"\end{tabular}")

In [ ]:
contingency_statement_research_confirmed = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_confirmed"],
)
contingency_full_statement_research_confirmed = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_confirmed"],
    margins=True,
)
contingency_full_statement_research_confirmed

In [ ]:
chi2_statement_research_confirmed = chi2_contingency(contingency_statement_research_confirmed)
chi2_statement_research_confirmed

In [ ]:
contingency_statement_research_helped = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_helped"],
)
contingency_full_statement_research_helped = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_helped"],
    margins=True,
)
contingency_full_statement_research_helped

In [ ]:
chi2_statement_research_helped = chi2_contingency(contingency_statement_research_helped)
chi2_statement_research_helped

In [ ]:
contingency_statement_research_not_helped = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_not_helped"],
)
contingency_full_statement_research_not_helped = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_not_helped"],
    margins=True,
)
contingency_full_statement_research_not_helped

In [ ]:
chi2_statement_research_not_helped = chi2_contingency(contingency_statement_research_not_helped)
chi2_statement_research_not_helped

In [ ]:
contingency_statement_research_learned = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_learned"],
)
contingency_full_statement_research_learned = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_learned"],
    margins=True,
)
contingency_full_statement_research_learned

In [ ]:
chi2_statement_research_learned = chi2_contingency(contingency_statement_research_learned)
chi2_statement_research_learned

In [ ]:
contingency_statement_research_continue = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_continue"],
)
contingency_full_statement_research_continue = crosstab(
    index=df_test2["quality"],
    columns=df_test2["statement_research_continue"],
    margins=True,
)
contingency_full_statement_research_continue

In [ ]:
chi2_statement_research_continue = chi2_contingency(contingency_statement_research_continue)
chi2_statement_research_continue

In [ ]:
contingency_statement_research_confirmed_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_confirmed"],
)
contingency_full_statement_research_confirmed_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_confirmed"],
    margins=True,
)
contingency_full_statement_research_confirmed_background

In [ ]:
chi2_statement_research_confirmed_background = chi2_contingency(contingency_statement_research_confirmed_background)
chi2_statement_research_confirmed_background

In [ ]:
contingency_statement_research_helped_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_helped"],
)
contingency_full_statement_research_helped_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_helped"],
    margins=True,
)
contingency_full_statement_research_helped_background

In [ ]:
chi2_statement_research_helped_background = chi2_contingency(contingency_statement_research_helped_background)
chi2_statement_research_helped_background

In [ ]:
contingency_statement_research_not_helped_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_not_helped"],
)
contingency_full_statement_research_not_helped_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_not_helped"],
    margins=True,
)
contingency_full_statement_research_not_helped_background

In [ ]:
chi2_statement_research_not_helped_background = chi2_contingency(contingency_statement_research_not_helped_background)
chi2_statement_research_not_helped_background

In [ ]:
contingency_statement_research_learned_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_learned"],
)
contingency_full_statement_research_learned_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_learned"],
    margins=True,
)
contingency_full_statement_research_learned_background

In [ ]:
chi2_statement_research_learned_background = chi2_contingency(contingency_statement_research_learned_background)
chi2_statement_research_learned_background

In [ ]:
contingency_statement_research_continue_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_continue"],
)
contingency_full_statement_research_continue_background = crosstab(
    index=df_test2["background"],
    columns=df_test2["statement_research_continue"],
    margins=True,
)
contingency_full_statement_research_continue_background

In [ ]:
chi2_statement_research_continue_background = chi2_contingency(contingency_statement_research_continue_background)
chi2_statement_research_continue_background

In [ ]:
tests = [
    (r"Background", contingency_full_statement_research_confirmed_background, chi2_statement_research_confirmed_background, contingency_full_statement_research_helped_background, chi2_statement_research_helped_background, contingency_full_statement_research_not_helped_background, chi2_statement_research_not_helped_background, contingency_full_statement_research_learned_background, chi2_statement_research_learned_background, contingency_full_statement_research_continue_background, chi2_statement_research_continue_background),
    (r"Quality", contingency_full_statement_research_confirmed, chi2_statement_research_confirmed, contingency_full_statement_research_helped, chi2_statement_research_helped, contingency_full_statement_research_not_helped, chi2_statement_research_not_helped, contingency_full_statement_research_learned, chi2_statement_research_learned, contingency_full_statement_research_continue, chi2_statement_research_continue),
]
alpha = 0.05
correction = 5
alpha_corrected = alpha / correction  # Bonferroni correction across outcomes
# TODO: Indicate change compared to expected frequencies.
# print(r"\newlength{\largetabcolsep}")
# print(r"\setlength{\largetabcolsep}{3\tabcolsep}")
print(r"\begin{tabular}{@{}l@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{\hspace{\largetabcolsep}}r@{ }lr@{ }lr@{}}")
print(r"  \toprule")
print(r"  \textbf{Predictor} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Confirm.\ prev.\ opinion}} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Could make better decis.}} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Results did not help}} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{\textbf{Learned something new}} & \multicolumn{5}{@{}c@{}}{\textbf{Would do further search}} \\")
print(r"  & \multicolumn{2}{@{}l}{No} & \multicolumn{2}{l}{Yes} & \(\sum\) & \multicolumn{2}{@{}l}{No} & \multicolumn{2}{l}{Yes} & \(\sum\) & \multicolumn{2}{@{}l}{No} & \multicolumn{2}{l}{Yes} & \(\sum\) & \multicolumn{2}{@{}l}{No} & \multicolumn{2}{l}{Yes} & \(\sum\) & \multicolumn{2}{@{}l}{No} & \multicolumn{2}{l}{Yes} & \(\sum\) \\")
for variable_name, contingency_1, chi2_result_1, contingency_2, chi2_result_2, contingency_3, chi2_result_3, contingency_4, chi2_result_4, contingency_5, chi2_result_5 in tests:
    statistic_1, p_value_1, dof_1, _ = chi2_result_1
    statistic_2, p_value_2, dof_2, _ = chi2_result_2
    statistic_3, p_value_3, dof_3, _ = chi2_result_3
    statistic_4, p_value_4, dof_4, _ = chi2_result_4
    statistic_5, p_value_5, dof_5, _ = chi2_result_5
    print(r"  \midrule")
    print(r"  \textit{" + variable_name + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_1, alpha_corrected) + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_2, alpha_corrected) + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_3, alpha_corrected) + r"} & \multicolumn{5}{@{}c@{\hspace{\largetabcolsep}}}{" + chi2_to_tex(chi2_result_4, alpha_corrected) + r"} & \multicolumn{5}{@{}c@{}}{" + chi2_to_tex(chi2_result_5, alpha_corrected) + r"} \\")
    print(r"  \midrule")
    for i, ((index_1, row_1), (index_2, row_2), (index_3, row_3), (index_4, row_4), (index_5, row_5)) in enumerate(zip(contingency_1.iterrows(), contingency_2.iterrows(), contingency_3.iterrows(), contingency_4.iterrows(), contingency_5.iterrows())):
        assert index_1 == index_2 == index_3 == index_4 == index_5
        index = index_1
        if "All" in index:
            index = r"\(\sum\)"
        columns = [
            index,
            f"{row_1['no']}",
            diff_expected_freq_tex(row_1, "no", index, i, chi2_result_1),
            f"{row_1['yes']}",
            diff_expected_freq_tex(row_1, "yes", index, i, chi2_result_1),
            f"{row_1['All']}",
            f"{row_2['no']}",
            diff_expected_freq_tex(row_2, "no", index, i, chi2_result_2),
            f"{row_2['yes']}",
            diff_expected_freq_tex(row_2, "yes", index, i, chi2_result_2),
            f"{row_2['All']}",
            f"{row_3['no']}",
            diff_expected_freq_tex(row_3, "no", index, i, chi2_result_3),
            f"{row_3['yes']}",
            diff_expected_freq_tex(row_3, "yes", index, i, chi2_result_3),
            f"{row_3['All']}",
            f"{row_4['no']}",
            diff_expected_freq_tex(row_4, "no", index, i, chi2_result_4),
            f"{row_4['yes']}",
            diff_expected_freq_tex(row_4, "yes", index, i, chi2_result_4),
            f"{row_4['All']}",
            f"{row_5['no']}",
            diff_expected_freq_tex(row_5, "no", index, i, chi2_result_5),
            f"{row_5['yes']}",
            diff_expected_freq_tex(row_5, "yes", index, i, chi2_result_5),
            f"{row_5['All']}",
        ]
        print(r"  " + r" & ".join(columns) + r" \\")
print(r"  \bottomrule")
print(r"\end{tabular}")

In [ ]:
alpha_corrected

## Correlations

In [ ]:
pearsonr(df_doc["rank"], df_doc["has_fully_read_document"])

In [ ]:
pearsonr(df_doc["rank"], df_doc["has_partially_read_document"])

In [ ]:
pearsonr(df_doc["rank"], df_doc["influence_document"])

In [ ]:
pearsonr(df_doc["rank"], df_doc["quality_score_document"])

In [ ]:
pearsonr(df_doc["rank"], df_doc["relevance_document"])

In [ ]:
pearsonr(df_doc["relevance_document"], df_doc["quality_score_document"])